In [30]:
''' This code generates an autoregressive time series with added normal noise given its order and initial values 
    and stores the data in the Data.xlsx file. 
    specify the row to store the data in variable r
'''
# set row in Data.xlsx file
r = 1
# set standard deviation
sigma = 0.1

import numpy as np
import pandas as pd
import openpyxl
import copy
# Get which order the user wants for the autoregressive model
order = int(input("Order for Autoregressive model"))

# create an array of shape(order, ) and input coefficients in it
coeff = np.zeros((order, ))
for idx in range(order) :
    coeff[idx] = float(input("coefficient : " + str(idx + 1)))
coeff.reshape((-1, 1))
const = float(input("coefficiet : "+ str(order+1)))
# print(coeff)

# get the initial values of the time series
initial = np.zeros((order, ))
for idx in range(order):
    initial[idx] = float(input("initial value at time " + str(idx + 1) + " : "))
# print (initial)

# get number of time steps
len = int(input("Length of time series to be generated : "))

# write initial data in the excel file
wb = openpyxl.load_workbook("Data.xlsx")
ws = wb.active

for idx in range(order):
    ws.cell(row=r, column=idx+1).value = initial[idx]

# generate the autoregressive time series of the given order
# latest stores the latest values of the times series
latest = copy.deepcopy(initial)

for idx in range(len):
    # multiply the matrices
    val = np.matmul(latest, coeff) + const + np.random.normal(0, sigma)
    np.roll(latest, -1)
    latest[order-1] = val
    ws.cell(row=r, column=order+idx+1).value = val

wb.save("Data.xlsx")